In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import cartopy.crs as ccrs
import cmocean
from cartopy.feature import LAND
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
import sys
sys.path.append("scripts/")
import processing as proc

In [ ]:
rho0 = 1026 # Reference density [kg/m^3]

In [ ]:
ref_lat = 40 # Reference latitude for AMOC calculations (°N)

In [ ]:
lowpass_cutoff = 70

### Import AMOC fields

In [ ]:
model_list = pd.read_csv("model_list.csv").set_index("Model")
sel_models = list(model_list.index)

In [ ]:
# Import streamfunction fields
amoc_strf = {}
amoc_strf_detr = {}
for model in sel_models:
    member = model_list.loc[model,"Member"]
    amoc_files = glob.glob("/home/omehling/work/cmip6/piControl_processed/amoc/CMIP6_{}_*{}*.nc".format(model, member))
    if len(amoc_files) != 1:
        raise ValueError("{} files found for {}".format(len(amoc_files), model))
    ds_imp = xr.open_dataset(amoc_files[0], use_cftime=True)
    # Rename "rlat" to "lat" if needed
    if 'rlat' in list(ds_imp.dims):
        ds_imp = ds_imp.rename({'rlat':'lat'})
    # Select streamfunction variable
    moc_var_name = "msftmz" if "msftmz" in list(ds_imp.data_vars) else "msftyz"
    amoc_strf[model] = ds_imp[moc_var_name].drop("sector")/rho0*1e-6 # Units: kg/s -> Sv
    amoc_strf_detr[model] = proc.detrend_xr(amoc_strf[model], 2) # Quadratic detrending
    
    print(model)

### Load data

Mixed layer depth (mlotst) + sea ice concentration (siconc) for March

In [ ]:
def load_cell_area(model_code):
    model_grid = "gn"
    if model_code == "CESM2": model_grid = "gr"
    
    areacello_path = glob.glob("/home/omehling/synda-CMIP/*/*/piControl/*/Ofx/areacello/{}/*/areacello*{}*.nc".format(model_grid,model_code.replace('_','*')))
    if len(areacello_path)==0:
        areacello_path = glob.glob("/home/omehling/synda-CMIP/*/*/historical/*/Ofx/areacello/{}/*/areacello*{}*.nc".format(model_grid,model_code.replace('_','*')))
    return xr.open_dataset(areacello_path[0])["areacello"]

In [ ]:
siconc = {}
siconc_detr = {}
mlotst = {}
mlotst_detr = {}

for i, model_id in enumerate(sel_models):
    member = model_list.loc[model_id,"Member"]
    
    # Load and detrend sic
    filname_sic = glob.glob("/home/omehling/work/cmip6/piControl_processed/siconc_mar/CMIP6_{}*{}*r360x180*.nc".format(model_id, member))
    if len(filname_sic) != 1:
        print("Warning: Skipping {} for siconc, {} files found instead of 1".format(model_id, len(filname_sic)))
        continue
    sic_load = xr.open_dataset(filname_sic[0], use_cftime=True)["siconc"]
    siconc[model_id] = sic_load
    siconc_detr[model_id] = proc.detrend_xr(sic_load, 2, keep_mean=True)
    siconc_detr[model_id] = siconc_detr[model_id].where(siconc_detr[model_id]>0, 0)
    siconc_detr[model_id] = siconc_detr[model_id].where(siconc_detr[model_id]<100, 100)
    
    # Load and detrend mlotst
    filname_mld = glob.glob("/home/omehling/work/cmip6/piControl_processed/mlotst/CMIP6_{}*{}*r360x180*.nc".format(model_id, member))
    if len(filname_mld) != 1:
        print("Warning: Skipping {} for mlotst, {} files found instead of 1".format(model_id, len(filname_mld)))
        continue
    mld_load = xr.open_dataset(filname_mld[0], use_cftime=True)["mlotst"]
    mlotst[model_id] = mld_load
    mlotst_detr[model_id] = proc.detrend_xr(mld_load, 2, keep_mean=True)
    mlotst_detr[model_id] = mlotst_detr[model_id].where(mlotst_detr[model_id]>0, 0)
    
    print(model_id)

Merge sea ice climatology

In [ ]:
siconc_mar_mean = {}
for model_id in sel_models:
    siconc_mar_mean[model_id] = siconc[model_id].mean("time")
siconc_mar_mean_xr = xr.concat(list(siconc_mar_mean.values()), dim="model")
siconc_mar_mean_xr["model"] = list(siconc_mar_mean.keys())

Observational/Paleo datasets + Bathymetry:

In [ ]:
# PaleoSST
paleosst_sic = []
for member_id in range(1,51):
    paleosst_sic_member = xr.open_dataset("~/data/reconstructions/PaleoSST/processed/PaleoSST_SIC_1000-1849_R{:03d}_marchmean.nc".format(member_id), use_cftime=True)
    paleosst_sic_member = paleosst_sic_member.squeeze().expand_dims({"member": 1})
    paleosst_sic_member["member"] = [member_id]
    paleosst_sic.append(paleosst_sic_member)
paleosst_sic = xr.concat(paleosst_sic, dim="member")["sic"]

In [ ]:
paleosst_sic_ensmean = paleosst_sic.mean("member").rename({"latitude": "lat", "longitude": "lon"})
paleosst_sic_ens05 = paleosst_sic.quantile(.05,"member").rename({"latitude": "lat", "longitude": "lon"})
paleosst_sic_ens95 = paleosst_sic.quantile(.95,"member").rename({"latitude": "lat", "longitude": "lon"})

In [ ]:
# "Sea ice back to 1850" (Walsh et al.) for comparison
sic_g10010_pi = xr.open_dataset("/home/omehling/data/obs/G10010_V2/G10010_sibt1850_v2.0_monmean_1850-1879.nc")["seaice_conc"].isel(time=2)
sic_g10010_pi = sic_g10010_pi.where(sic_g10010_pi<=110).rename({"latitude": "lat", "longitude": "lon"})

In [ ]:
# Bathymetry
bathy = xr.open_dataset("/home/omehling/data/bathymetry/GEBCO_2023_01deg.nc")["elevation"]
bathy = proc.ds_to_180(bathy.where(bathy<=0))

### Sea ice + mixed layer composites (Fig. 3a-d, S8)

In [ ]:
for model_id in sel_models:
    amoc_index = amoc_strf_detr[model_id].sel(lat=ref_lat, method="nearest").sel(lev=slice(500,None)).max("lev")
    amoc_index_lf = proc.filter_xr(amoc_index, 70, detrend=False)

    # Calc composites
    idx_amoc_pos = np.where(amoc_index_lf>amoc_index_lf.mean()+amoc_index_lf.std())[0] # strong AMOC
    idx_amoc_neg = np.where(amoc_index_lf<amoc_index_lf.mean()-amoc_index_lf.std())[0] # weak AMOC

    comp_strong_sic = siconc_detr[model_id].isel(time=idx_amoc_pos).mean('time')
    comp_strong_mld = mlotst_detr[model_id].isel(time=idx_amoc_pos).mean('time')
    comp_weak_sic = siconc_detr[model_id].isel(time=idx_amoc_neg).mean('time')
    comp_weak_mld = mlotst_detr[model_id].isel(time=idx_amoc_neg).mean('time')

    # Plot
    fig,axes = plt.subplots(2,1,figsize=(6,5),subplot_kw=dict(projection=ccrs.Miller()))
    ice_cmap = sns.color_palette("ch:s=.05,rot=-.25", 12)[:-2]
    mask_regrid = ~siconc[model_id].isel(time=0, drop=True).isnull()
    if model_id == "CESM2":
        mask_regrid.values[:,320]=mask_regrid.values[:,321] # fix processing error for mask

    ax = axes.flat[0]
    c = proc.ds_to_180(comp_strong_sic.where(mask_regrid)).plot.contourf(ax=ax, transform=ccrs.PlateCarree(),add_colorbar=False,levels=np.arange(0,110,10),colors=ice_cmap)
    cs = proc.ds_to_180(comp_strong_mld.where(mask_regrid)).plot.contour(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(200,2700,500),colors='k',linewidths=.8)
    ax.set_extent([-80, 35, 40, 80], ccrs.PlateCarree())
    ax.coastlines(lw=.4,color='C7')
    plt.clabel(cs, [200,700,1200,1700], fontsize=7.5, inline_spacing=1)
    ax.set(title=model_id+': strong AMOC')

    ax=axes.flat[1]
    c = proc.ds_to_180(comp_weak_sic.where(mask_regrid)).plot.contourf(ax=ax, transform=ccrs.PlateCarree(),add_colorbar=False,levels=np.arange(0,110,10),colors=ice_cmap)
    cs = proc.ds_to_180(comp_weak_mld.where(mask_regrid)).plot.contour(ax=ax, transform=ccrs.PlateCarree(),levels=np.arange(200,2700,500),colors='k',linewidths=.8)
    ax.set_extent([-80, 35, 40, 80], ccrs.PlateCarree())
    ax.set(title=model_id+': weak AMOC')
    ax.coastlines(lw=.4,color='C7')
    plt.clabel(cs, [200,700,1200,1700], fontsize=7.5, inline_spacing=2)

    cbar_ax = fig.add_axes([0.80, 0.2, 0.015, 0.6])
    cbar = fig.colorbar(c, cax=cbar_ax, extend='both', ticks=np.arange(0,120,20))
    cbar.set_label('Sea ice concentration [%]', fontsize=11)

    fig.savefig("figures/"+model_id+"-seaice-mld.pdf", bbox_inches='tight', dpi=300)
    fig.savefig("figures/png/"+model_id+"-seaice-mld.png", bbox_inches='tight', dpi=300)
    
    print(model_id)

### Sea ice bias (Fig. 3e)

In [ ]:
bathy_sel = bathy.sel(lat=slice(37,83), lon=slice(-83,58))

In [ ]:
fig,ax = plt.subplots(figsize=(6,3),subplot_kw=dict(projection=ccrs.Miller()))
ice_edge_threshold = 15

#paleosst_si_edge_uncertainty = np.logical_and(paleosst_sic_ens95>.3, paleosst_sic_ens05<.3).astype(int)
#paleosst_si_edge_uncertainty.where(paleosst_si_edge_uncertainty).plot(ax=ax,transform=ccrs.PlateCarree(), levels=[0.5], colors=["#aaaaaa"], add_colorbar=False)
line_colors = sns.color_palette("colorblind", 9)
for i, model_id in enumerate(siconc_mar_mean_xr.model):
    proc.ds_to_180(siconc_mar_mean_xr.sel(model=model_id)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),levels=[ice_edge_threshold],linewidths=1, colors=[line_colors[i]])#colors="C{}".format(i))
proc.ds_to_180(paleosst_sic_ensmean).plot.contour(ax=ax,transform=ccrs.PlateCarree(),levels=[ice_edge_threshold/100],linewidths=2,colors="k")
proc.ds_to_180(sic_g10010_pi).plot.contour(ax=ax,transform=ccrs.PlateCarree(),levels=[ice_edge_threshold],linewidths=2,colors="C7")

cbat = bathy.sel(lat=slice(37,83), lon=slice(-83,58)).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=[-3000,-2000,-1250,-1000,-650,-200,0], colors=sns.color_palette("Blues",24,desat=0.7)[0:15:2][::-1], add_colorbar=False)
ax.pcolormesh([-83,58],[37,83],[[1]],cmap="Greys",alpha=.2,transform=ccrs.PlateCarree(), zorder=1.8)

ax.add_feature(LAND, facecolor='#fafafa', zorder=2.5)
ax.coastlines(color="#bbbbbb", lw=.3, zorder=2.8)
ax.set_extent([-80, 55, 40, 80], ccrs.PlateCarree())
ax.set_title("")
handles = [plt.Line2D([0,1],[0,1],color=col,lw=1) for col in line_colors]
handles.append(plt.Line2D([0,1],[0,1],color="k",lw=2))
handles.append(plt.Line2D([0,1],[0,1],color="C7",lw=2))
labels = list(siconc_mar_mean_xr.model.values)+["PaleoSST", "SIBT1850"]

divider = make_axes_locatable(ax)
cax = divider.new_horizontal(size="2%", pad=0.2, axes_class=plt.Axes)
fig.add_axes(cax)
cbar = fig.colorbar(cbat, cax=cax)
cbar.set_label(label="Bathymetry [m]", size=8.5)
cbar.set_ticks(ticks=cbar.get_ticks(), labels=[int(i*-1) for i in cbar.get_ticks()])
cax.tick_params(labelsize=8.5)
ax.spines['geo'].set_linewidth(0.5)

fig.legend(handles, labels, ncols=4, bbox_to_anchor=(0.5,0.1), loc="upper center", frameon=False, columnspacing=0.95);

fig.savefig("figures/Sea_ice_bias_PI.pdf",bbox_inches="tight")
fig.savefig("figures/png/Sea_ice_bias_PI.png",bbox_inches="tight",dpi=300)